In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
import csv

Using device: cpu
Model loaded.
🔍 Evaluating ecg_record_100.png...
✅ Saved: cae\ecg_record_100.png
🔍 Evaluating ecg_record_101.png...
✅ Saved: cae\ecg_record_101.png
🔍 Evaluating ecg_record_102.png...
✅ Saved: cae\ecg_record_102.png
🔍 Evaluating ecg_record_103.png...
✅ Saved: cae\ecg_record_103.png
🔍 Evaluating ecg_record_104.png...
✅ Saved: cae\ecg_record_104.png
🔍 Evaluating ecg_record_105.png...
✅ Saved: cae\ecg_record_105.png
🔍 Evaluating ecg_record_106.png...
✅ Saved: cae\ecg_record_106.png
🔍 Evaluating ecg_record_107.png...
✅ Saved: cae\ecg_record_107.png
🔍 Evaluating ecg_record_108.png...
✅ Saved: cae\ecg_record_108.png
🔍 Evaluating ecg_record_109.png...
✅ Saved: cae\ecg_record_109.png
🔍 Evaluating ecg_record_111.png...
✅ Saved: cae\ecg_record_111.png
🔍 Evaluating ecg_record_112.png...
✅ Saved: cae\ecg_record_112.png
🔍 Evaluating ecg_record_113.png...
✅ Saved: cae\ecg_record_113.png
🔍 Evaluating ecg_record_114.png...
✅ Saved: cae\ecg_record_114.png
🔍 Evaluating ecg_record_115.pn

In [ ]:
# Define Convolutional Autoencoder
class ECG_CAE(nn.Module):
    def __init__(self):
        super(ECG_CAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 32, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        _, _, h, w = x.size()
        decoded = decoded[:, :, :h, :w]
        return encoded, decoded



In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((296, 1024)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])



In [ ]:
# Dataloader
def get_dataloader(data_dir, batch_size=16, shuffle=True):
    dataset = ImageFolder(data_dir, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=True)

# Train function
def train_model(model, dataloader, device, num_epochs=10, lr=1e-3, save_path="ecg_cae.pth"):
    model.train()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        total_loss = 0.0
        for images, _ in dataloader:
            images = images.to(device, non_blocking=True)
            _, outputs = model(images)
            loss = criterion(outputs, images)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.6f}")

    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

# Evaluate single image and return duration
def evaluate_image(model, image_path, device, save_dir="cae"):
    model.eval()
    image = Image.open(image_path).convert("L")
    resized = transform(image).unsqueeze(0).to(device)

    start_time = time.time()
    with torch.no_grad():
        encoded, decoded = model(resized)
    end_time = time.time()
    duration = end_time - start_time

    output_img = decoded.cpu().squeeze(0).squeeze(0).numpy()
    output_img = (output_img * 0.5) + 0.5
    output_img = np.clip(output_img, 0, 1)
    output_pil = Image.fromarray((output_img * 255).astype(np.uint8))

    # Save with same filename in "cae" folder
    os.makedirs(save_dir, exist_ok=True)
    filename = os.path.basename(image_path)
    output_path = os.path.join(save_dir, filename)
    output_pil.save(output_path)

    print(f"✅ Saved: {output_path}")
    return duration

# Evaluate a folder of images and save durations to CSV
def evaluate_folder(model, folder_path, device, output_csv='cae_durations.csv', save_dir="cae"):
    durations = []
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for fname in image_files:
        image_path = os.path.join(folder_path, fname)
        print(f"🔍 Evaluating {fname}...")
        duration = evaluate_image(model, image_path, device, save_dir)
        durations.append((fname, duration))

    # Save durations to CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Image Name', 'Duration (seconds)'])
        writer.writerows(durations)

    print(f"✅ Durations saved to: {output_csv}")



In [ ]:

if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = ECG_CAE().to(device)

    if not os.path.exists("ecg_cae.pth"):
        data_dir = "org_train"
        dataloader = get_dataloader(data_dir, batch_size=16, shuffle=True)
        train_model(model, dataloader, device, num_epochs=40)

    if os.path.exists("ecg_cae.pth"):
        model.load_state_dict(torch.load("ecg_cae.pth", map_location=device))
        print("Model loaded.")

    # Evaluate images in folder and save durations
    evaluate_folder(model, folder_path="org_ecg_10sec_resize", device=device, output_csv="cae_durations.csv", save_dir="cae")
